In [86]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from itertools import product, chain
import functools 
import operator 
from csv import reader
import regex as re

# Load the data

In [61]:
def features_into_array(path):
    with open(path, 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(csv_reader)
        X = list()
        if header != None:
            for row in csv_reader:
                # row variable is a list that represents a row in csv
                X.append(np.array(row[1]))
                
    X = np.array(X) ## dtype might be changed in something more convenient. For now, dtype = "<U1"
    return X

In [62]:
Xtr0 = features_into_array("data/Xtr0.csv")
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1 = features_into_array("data/Xtr1.csv")
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2 = features_into_array("data/Xtr2.csv")
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)

In [142]:
def accuracy(y_true,y_pred, mode='SVM'):
    n = y_true.shape[0]
    if mode == 'SVM':
        predictions = np.ones(n)
        predictions[y_pred < 0] = 0
    else:
        predictions = np.zeros(n)
        predictions[y_pred >= 0.5] = 1
    
    return np.sum(y_true == predictions) / n

In [62]:
Xtr0[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

## Spectrum kernel

For a fixed value k (that needs to be tuned), the k-spectrum kernel is defined as : 


\begin{align*}
K(x,x^{\prime}) := \sum_{u \in \mathcal{A}^k} \phi_{u}(x) \phi_{u}(x^{\prime})
\end{align*}

In [38]:
def all_possible_substrings(k):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = list()
    for i in alphabet_tuples:
        alphabet.append(functools.reduce(operator.add, (i)))
    return alphabet

In [18]:
def neighbors(word, m):
    """
    This gives neighbors that differ in exactly m places
    """
    
    char_list = list(['A', 'C','G','T'])
    assert(m <= len(word))

    if m == 0:
        return [word]

    r2 = neighbors(word[1:], m-1)
    r = [c + r3 for r3 in r2 for c in char_list if c != word[0]]

    if (m < len(word)):
        r2 = neighbors(word[1:], m)
        r += [word[0] + r3 for r3 in r2]

    return r

def neighbors2(pattern, m):
    """
    This gives neighbors that differ in at most m places.
    """
    return sum([neighbors(pattern, d2) for d2 in range(m + 1)], [])


In [57]:
all_possible_substrings_mismatch(3,1)

['AAA', 'CAA', 'GAA', 'TAA', 'ACA', 'AGA', 'ATA', 'AAC', 'AAG', 'AAT']
['AAC', 'CAC', 'GAC', 'TAC', 'ACC', 'AGC', 'ATC', 'AAA', 'AAG', 'AAT']
['AAG', 'CAG', 'GAG', 'TAG', 'ACG', 'AGG', 'ATG', 'AAA', 'AAC', 'AAT']
['AAT', 'CAT', 'GAT', 'TAT', 'ACT', 'AGT', 'ATT', 'AAA', 'AAC', 'AAG']
['ACA', 'CCA', 'GCA', 'TCA', 'AAA', 'AGA', 'ATA', 'ACC', 'ACG', 'ACT']
['ACC', 'CCC', 'GCC', 'TCC', 'AAC', 'AGC', 'ATC', 'ACA', 'ACG', 'ACT']
['ACG', 'CCG', 'GCG', 'TCG', 'AAG', 'AGG', 'ATG', 'ACA', 'ACC', 'ACT']
['ACT', 'CCT', 'GCT', 'TCT', 'AAT', 'AGT', 'ATT', 'ACA', 'ACC', 'ACG']
['AGA', 'CGA', 'GGA', 'TGA', 'AAA', 'ACA', 'ATA', 'AGC', 'AGG', 'AGT']
['AGC', 'CGC', 'GGC', 'TGC', 'AAC', 'ACC', 'ATC', 'AGA', 'AGG', 'AGT']
['AGG', 'CGG', 'GGG', 'TGG', 'AAG', 'ACG', 'ATG', 'AGA', 'AGC', 'AGT']
['AGT', 'CGT', 'GGT', 'TGT', 'AAT', 'ACT', 'ATT', 'AGA', 'AGC', 'AGG']
['ATA', 'CTA', 'GTA', 'TTA', 'AAA', 'ACA', 'AGA', 'ATC', 'ATG', 'ATT']
['ATC', 'CTC', 'GTC', 'TTC', 'AAC', 'ACC', 'AGC', 'ATA', 'ATG', 'ATT']
['ATG'

[]

In [77]:
def all_possible_substrings_mismatch(k,m):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = list()
    for i in alphabet_tuples:
        word = functools.reduce(operator.add, (i))
        l= [word]+neighbors2(word,m)[1:]
        alphabet.append(l)
    return alphabet

In [83]:
## TODO : a function that computes occurences 
## with overlapping option without calling regex if we have remaining time (lol)

def pre_indexing_by_sequence_mismatch(x, k, m):
    alphabet = all_possible_substrings_mismatch(k, m)
    d = dict()
    for letters in alphabet :
        cnt = 0
        for letter in letters:
            cnt += len(re.findall(letter, x, overlapped=True))
        d[letters[0]] = cnt
    return d

In [82]:
Xtr0[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

In [84]:
pre_indexing_by_sequence_mismatch(Xtr0[0],3,1)

{'AAA': 11,
 'AAC': 17,
 'AAG': 13,
 'AAT': 7,
 'ACA': 19,
 'ACC': 18,
 'ACG': 13,
 'ACT': 17,
 'AGA': 13,
 'AGC': 16,
 'AGG': 17,
 'AGT': 12,
 'ATA': 11,
 'ATC': 10,
 'ATG': 17,
 'ATT': 8,
 'CAA': 16,
 'CAC': 19,
 'CAG': 19,
 'CAT': 16,
 'CCA': 20,
 'CCC': 26,
 'CCG': 21,
 'CCT': 14,
 'CGA': 14,
 'CGC': 20,
 'CGG': 19,
 'CGT': 15,
 'CTA': 12,
 'CTC': 18,
 'CTG': 19,
 'CTT': 11,
 'GAA': 15,
 'GAC': 18,
 'GAG': 16,
 'GAT': 13,
 'GCA': 14,
 'GCC': 24,
 'GCG': 21,
 'GCT': 16,
 'GGA': 19,
 'GGC': 19,
 'GGG': 24,
 'GGT': 18,
 'GTA': 12,
 'GTC': 20,
 'GTG': 19,
 'GTT': 11,
 'TAA': 6,
 'TAC': 13,
 'TAG': 10,
 'TAT': 10,
 'TCA': 18,
 'TCC': 16,
 'TCG': 14,
 'TCT': 14,
 'TGA': 16,
 'TGC': 20,
 'TGG': 20,
 'TGT': 17,
 'TTA': 3,
 'TTC': 11,
 'TTG': 17,
 'TTT': 8}

In [95]:
all_possible_substrings_mismatch(3,1)

[['AAA', 'CAA', 'GAA', 'TAA', 'ACA', 'AGA', 'ATA', 'AAC', 'AAG', 'AAT'],
 ['AAC', 'CAC', 'GAC', 'TAC', 'ACC', 'AGC', 'ATC', 'AAA', 'AAG', 'AAT'],
 ['AAG', 'CAG', 'GAG', 'TAG', 'ACG', 'AGG', 'ATG', 'AAA', 'AAC', 'AAT'],
 ['AAT', 'CAT', 'GAT', 'TAT', 'ACT', 'AGT', 'ATT', 'AAA', 'AAC', 'AAG'],
 ['ACA', 'CCA', 'GCA', 'TCA', 'AAA', 'AGA', 'ATA', 'ACC', 'ACG', 'ACT'],
 ['ACC', 'CCC', 'GCC', 'TCC', 'AAC', 'AGC', 'ATC', 'ACA', 'ACG', 'ACT'],
 ['ACG', 'CCG', 'GCG', 'TCG', 'AAG', 'AGG', 'ATG', 'ACA', 'ACC', 'ACT'],
 ['ACT', 'CCT', 'GCT', 'TCT', 'AAT', 'AGT', 'ATT', 'ACA', 'ACC', 'ACG'],
 ['AGA', 'CGA', 'GGA', 'TGA', 'AAA', 'ACA', 'ATA', 'AGC', 'AGG', 'AGT'],
 ['AGC', 'CGC', 'GGC', 'TGC', 'AAC', 'ACC', 'ATC', 'AGA', 'AGG', 'AGT'],
 ['AGG', 'CGG', 'GGG', 'TGG', 'AAG', 'ACG', 'ATG', 'AGA', 'AGC', 'AGT'],
 ['AGT', 'CGT', 'GGT', 'TGT', 'AAT', 'ACT', 'ATT', 'AGA', 'AGC', 'AGG'],
 ['ATA', 'CTA', 'GTA', 'TTA', 'AAA', 'ACA', 'AGA', 'ATC', 'ATG', 'ATT'],
 ['ATC', 'CTC', 'GTC', 'TTC', 'AAC', 'ACC', 'AGC', 

In [100]:
def pre_indexing_mismatch(X, k,m):
    """
    Transforms an input array into a sparse matrix encoding the number of occurences of each letter of
    the alphabet composed of substrings of size k
    """
    i = 0
    n = X.shape[0]
    alphabet = all_possible_substrings_mismatch(k,m)
    D = np.zeros((n,len(alphabet)))
    for x in X:
        d = dict()
        for letters in alphabet :
            cnt = 0
            for letter in letters:
                cnt += len(re.findall(letter, x, overlapped=True))
            d[letters[0]] = cnt
        data = np.array(list(d.items()))
        D[i] = data[:,1]
        i+=1
    D = csr_matrix(D, dtype = int)
    return D

In [101]:
pre_indexing_mismatch(Xtr0[:5],3,1).toarray()

array([[11, 17, 13,  7, 19, 18, 13, 17, 13, 16, 17, 12, 11, 10, 17,  8,
        16, 19, 19, 16, 20, 26, 21, 14, 14, 20, 19, 15, 12, 18, 19, 11,
        15, 18, 16, 13, 14, 24, 21, 16, 19, 19, 24, 18, 12, 20, 19, 11,
         6, 13, 10, 10, 18, 16, 14, 14, 16, 20, 20, 17,  3, 11, 17,  8],
       [20, 19, 10, 19, 17, 15,  8, 20, 11, 16,  9, 14, 22, 18, 17, 26,
        18, 12, 12, 19, 18, 19, 11, 15, 12,  6,  2, 13, 19, 18, 12, 25,
        10,  9, 11, 14,  9,  9,  3, 15,  5,  2,  2, 13, 18, 16, 10, 19,
        20, 20, 17, 31, 18, 21, 12, 27, 16, 12,  9, 23, 28, 25, 20, 34],
       [36, 24, 26, 26, 24, 15, 10, 19, 27, 14, 16, 17, 26, 15, 14, 18,
        25, 17, 17, 17, 18,  9,  6,  9, 10,  9,  9, 10, 18,  7, 12, 14,
        24, 13, 15, 13, 21,  7, 12, 11, 12, 14, 14, 13, 17, 13, 12, 14,
        28, 14, 16, 16, 14, 11, 10, 11, 16, 13, 13, 12, 17, 12, 17, 11],
       [32, 22, 22, 22, 20, 12, 12, 16, 23, 16, 18, 17, 19, 10, 18, 18,
        24, 14, 16, 13, 15, 13, 11,  8, 16,  7, 13,  9, 12, 1

In [195]:
## example

Xtr0[:5]
Dtr = pre_indexing(Xtr0[:5],2).toarray()
Dval = pre_indexing(Xtr0[15:19],2).toarray()

print("train ", Dtr)
print("")
print("val ", Dval)
print("")
print(np.inner(Dtr, Dtr))


train  [[ 4  8  4  4 10 11  4  6  6  7 10  7  0  6 12  1]
 [ 7  5  6  9  6  9  0  8  3  1  0  7 11  8  5 15]
 [18  6  8  6 10  3  0  5  3  6  6  6  8  3  6  6]
 [16  5  8  4  8  6  1  3  7  3  8  7  3  3  8 10]
 [11  4 11  4  6  5  1  9  8  6  9  4  4  6  7  5]]

val  [[ 1  5 11  4 11 10  0  9  5  6  2  7  3  9  7 10]
 [ 8  9  6  1 10 11  2  8  6  5 11  5  0  7  7  4]
 [ 9  3 12  4 11  6  1  3  7  7 15  6  1  5  7  3]
 [ 6  5  6  6 10 11  1 10  4  9  7  3  3  8  8  3]]

[[800 532 597 636 642]
 [532 866 674 667 612]
 [597 674 856 789 714]
 [636 667 789 824 716]
 [642 612 714 716 740]]


In [143]:
#def spectrum_function(x,y,k):
#    phi_x = pre_indexing(x, k)
#    phi_y = pre_indexing(y, k)
#    
#    merge_dict = {k: phi_x.get(k, 0) * phi_y.get(k, 0) for k in set(phi_x)}
#    return sum(merge_dict.values())


## TODO ##
def spectrum_kernel(X_train, X_val, k, mode="train"):
    n_train = X_train.shape[0]
    n_val = X_val.shape[0]
    
    diag_train, diag_val = np.zeros(n_train), np.zeros(n_val)
    
    for i in range(n_train):
        diag_train[i] = spectrum_function(X_train[i], X_train[i],k)
        
    for i in range(n_val):
        diag_val[i] = spectrum_function(X_train[i], X_val[i],k)
        
    K_train = diag_train * np.eye(n_train) # Computation along the diagonal 
    K_val = diag_val * np.eye(n_val) # Computation along the diagonal 
    
    if mode=="test":
        for i in range(n_val):
            for j in range(n_train):
                val = spectrum_function(X_val[i], X_train[j], k)
                K_val[i,j] = val
        return(K_val)
    
    else:
        for i in range(n_train):
            for j in range(i+1,n_train):
                val = spectrum_function(X_train[i], X_train[j], k)
                K_train[i,j] = val
                K_train[j,i] = val
        return(K_train)

In [130]:
## example

x = pre_indexing_by_sequence(Xtr0[0], 3)
print("x ", x)
print("")
y = pre_indexing_by_sequence(Xtr0[1], 3)
print('y ', y)

print('')
print(({k: x.get(k, 0) * y.get(k, 0) for k in x}.values()))


x  {'AAA': 2, 'AAC': 0, 'AAG': 0, 'AAT': 2, 'ACA': 3, 'ACC': 3, 'ACG': 2, 'ACT': 0, 'AGA': 2, 'AGC': 0, 'AGG': 1, 'AGT': 1, 'ATA': 0, 'ATC': 2, 'ATG': 2, 'ATT': 0, 'CAA': 2, 'CAC': 5, 'CAG': 2, 'CAT': 1, 'CCA': 3, 'CCC': 3, 'CCG': 1, 'CCT': 3, 'CGA': 1, 'CGC': 1, 'CGG': 2, 'CGT': 0, 'CTA': 0, 'CTC': 2, 'CTG': 4, 'CTT': 0, 'GAA': 0, 'GAC': 3, 'GAG': 2, 'GAT': 1, 'GCA': 4, 'GCC': 2, 'GCG': 0, 'GCT': 1, 'GGA': 1, 'GGC': 4, 'GGG': 3, 'GGT': 2, 'GTA': 0, 'GTC': 1, 'GTG': 5, 'GTT': 1, 'TAA': 0, 'TAC': 0, 'TAG': 0, 'TAT': 0, 'TCA': 0, 'TCC': 3, 'TCG': 1, 'TCT': 2, 'TGA': 2, 'TGC': 2, 'TGG': 4, 'TGT': 4, 'TTA': 0, 'TTC': 0, 'TTG': 1, 'TTT': 0}

y  {'AAA': 1, 'AAC': 1, 'AAG': 3, 'AAT': 2, 'ACA': 1, 'ACC': 4, 'ACG': 0, 'ACT': 0, 'AGA': 3, 'AGC': 0, 'AGG': 0, 'AGT': 3, 'ATA': 3, 'ATC': 4, 'ATG': 0, 'ATT': 2, 'CAA': 2, 'CAC': 2, 'CAG': 1, 'CAT': 1, 'CCA': 3, 'CCC': 2, 'CCG': 0, 'CCT': 4, 'CGA': 0, 'CGC': 0, 'CGG': 0, 'CGT': 0, 'CTA': 4, 'CTC': 1, 'CTG': 1, 'CTT': 1, 'GAA': 1, 'GAC': 0, 'GAG': 0, '